In [ ]:
import torch
import numpy as np
import torch.nn as nn
from scipy.io import loadmat
import matplotlib.pyplot as plt
from Dataset import Dataset, ELM, device
from torch.utils.data import DataLoader
torch.set_default_dtype(torch.float64)

In [ ]:
train = loadmat('data_training_reactor_pol.mat')
test = loadmat('data_validation_reactor_pol.mat')

y_train = 0.0001*(train['NAMW'][0]-train['NAMW'][0][0])
y_test = 0.0001*(test['NAMW'][0]-test['NAMW'][0][0])

x_train = np.concatenate((100*np.concatenate(([[0]], (train['FI'].T[1:]-train['FI'].T[0])), axis=0),
                          100*np.concatenate(([[0], [0]], (train['FI'].T[2:]-train['FI'].T[0])), axis=0),
                          np.concatenate(([0], y_train[1:]), axis=0).reshape(-1, 1),
                          np.concatenate(([0, 0], y_train[2:]), axis=0).reshape(-1, 1)), axis=1)

x_test = np.concatenate((100*np.concatenate(([[0]], (test['FI'].T[1:]-test['FI'].T[0])), axis=0),
                         100*np.concatenate(([[0], [0]], (test['FI'].T[2:]-test['FI'].T[0])), axis=0),
                         np.concatenate(([0], y_test[1:]), axis=0).reshape(-1, 1),
                         np.concatenate(([0, 0], y_test[2:]), axis=0).reshape(-1, 1)), axis=1)

In [ ]:
intersize = 400

x_inter = x_test[0:intersize]
y_inter = y_test[0:intersize]

x_test = x_test[intersize:]
y_test = y_test[intersize:]

In [ ]:
trainset = Dataset(x_train, y_train)
testset = Dataset(x_test, y_test)
interset = Dataset(x_inter, y_inter)

trainloader = DataLoader(trainset, batch_size=256, shuffle=True)
testloader = DataLoader(testset, batch_size=256, shuffle=False)

activation = nn.Sigmoid()

In [ ]:
model = ELM(4, intersize, 1).to(device)

In [ ]:
V = activation(model.fc1.bias.data.reshape(-1, 1) + model.fc1.weight.data @ interset[:][0].T.to(device))
H = torch.concatenate([torch.ones(1, len(interset)).to(device), V.to(device)], dim=0).T
T = interset[:][1].reshape(1, -1).T.to(device)
W2 = torch.linalg.pinv(H) @ T.reshape(1, -1, 1)
model.fc2.bias.data, model.fc2.weight.data = W2[0][0], W2[0][1:].T

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-9, foreach=False)

In [ ]:
num_epochs = 100
all_losses = []
model.train()

for epoch in range(num_epochs):
    for i, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        x, y = x.reshape(-1, 4).to(device), y.reshape(-1, 1).to(device)
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
    all_losses.append(loss.item())
    print(f'Epoch: [{epoch+1}/{num_epochs}], loss: {loss.item():.6f}')

plt.plot(all_losses)
# plt.yscale('log')
plt.title('Uczenie modelu')
plt.xlabel('Epoki uczące')
plt.ylabel('Błąd średniokwadratowy')
plt.grid()
plt.show()

In [ ]:
print(f'Train error: {(model(trainset[:][0].clone().detach().reshape(-1, 4).to(device))-trainset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean()}')
print(f'Test error: {(model(testset[:][0].clone().detach().reshape(-1, 4).to(device))-testset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean()}')
plt.plot(y_test)
plt.plot(model(torch.tensor(x_test).reshape(-1, 4).to(device)).cpu().detach().numpy(), 'r--')
plt.legend(['Dane weryfikujące', 'Model'], loc='upper right')
plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.show()

In [ ]:
plt.plot([np.min(y_test), np.max(y_test)], [np.min(y_test), np.max(y_test)])
plt.plot(y_test, model(torch.tensor(x_test).reshape(-1, 4).to(device)).cpu().detach().numpy(), '.', markersize=1)
plt.legend(['Dane', 'Model'], loc='upper left')
plt.xlabel('Dane')
plt.ylabel('Model')
plt.grid()
plt.show()